<h2> Confidence Intervals for Sample Tiers (Downstream) </h2>

In [1]:
# from IPython.core.display import display, HTML
from IPython.display import HTML
from IPython.core.magic import register_cell_magic

@register_cell_magic
def toggle_code(self, cell):
    display(HTML('''
        <script>code_show=true;
        function code_toggle() {
            if (code_show) { $('div.input').hide();}
            else {$('div.input').show();}
            code_show = !code_show
        }
        $(document).ready(code_toggle);
        </script>
        <button onClick="javascript:code_toggle()">Toggle code</button>'''
    ))
    return display(HTML('<p>Click on the button above to show/hide code.</p>'))

In [139]:
%%toggle_code
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as st
from scipy.stats import norm
from google.cloud import bigquery

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/andreasfreund/.config/gcloud/application_default_credentials.json"
%load_ext google.cloud.bigquery
client = bigquery.Client()

#setting max rows to print
pd.set_option('display.max_rows', 150)

# style
plt.style.use("fivethirtyeight")
%matplotlib inline




print('Executed')

<h3>Helper Functions</h3>

In [159]:
def get5th(downloadTier, isp):
    
#     isp = "'"+isp+"'"

    fifthQuery = """
        SELECT 
        mt.download,
        mt.ISP,
        mt.Technology,
        APPROX_QUANTILES(httpmt.bytes_sec/125000, 100)[OFFSET(5)] as fifthPct,
        COUNT(DISTINCT httpmt.unit_id) as totalBoxes,
        FROM `broadband-data.mba_data.curr_httpgetmt` as httpmt 
            INNER JOIN`broadband-data.mba_data.unit-profile-sept2020` as mt
            ON httpmt.unit_id=mt.unit_id
        WHERE mt.download = {} and mt.ISP = {}
        AND (EXTRACT(HOUR FROM httpmt.dtime) >= 19) AND (EXTRACT(HOUR FROM httpmt.dtime) <=23) AND
                httpmt.ddate < (CAST('2020-10-01' as DATE))
        GROUP BY mt.download, mt.ISP, mt.Technology
        """.format(downloadTier, isp)
    df= client.query(fifthQuery).to_dataframe()
#     print(df)
    
    return(float(df['fifthPct']))

In [160]:
def ispTierCI(downloadTier, isp):
    import scipy.stats as st
    
    isp = "'"+isp+"'"
    
    result = """
    SELECT 
        mt.ISP,
        mt.Technology, 
        mt.Download, 
        mt.unit_id as unitId,
        httpmt.dtime as time,
        httpmt.ddate as date, 
        httpmt.bytes_sec/125000 as Mbps, 
    FROM
        `broadband-data.mba_data.curr_httpgetmt` as httpmt
        INNER JOIN
        `broadband-data.mba_data.unit-profile-sept2020` as mt
        ON httpmt.unit_id=mt.Unit_ID
    WHERE (EXTRACT(HOUR FROM httpmt.dtime) >= 19) AND (EXTRACT(HOUR FROM httpmt.dtime) <=23) AND
                httpmt.ddate < (CAST('2020-10-01' as DATE)) AND mt.ISP = {} AND mt.Download = {}
    ORDER BY Mbps
    """.format(isp, downloadTier)
    
    result = client.query(result).to_dataframe()
    tech = (result.iloc[0]['Technology'])
    
#     result['pct-of-advertised'] = (result['Download']/result['Mbps'] * 100) 

    
#     print("95% Confidence Interval (Mbps)",
#           st.norm.interval(alpha=0.95, loc=np.mean(result['Mbps']), 
#                            scale=st.sem(result['Mbps'])))
    
#     print("95% Confidence Interval (normalized to % of advertised speed)",
#           st.norm.interval(alpha=0.95, loc=np.mean(result['pct-of-advertised']), 
#                            scale=st.sem(result['pct-of-advertised'])))
    
    print(("Getting results for {} {} download tier: {} Mbps...").format(isp, tech, downloadTier))
    
    print("95% Confidence Interval (Mbps)",
          st.t.interval(0.95, len(result['Mbps']-1), loc=np.mean(result['Mbps']),
                                         scale=st.sem(result['Mbps'])))
    
    _95ConsistentSpeed = get5th(downloadTier, isp)
    print("95th Consistent Speed for this tier and ISP is", _95ConsistentSpeed)
    
    finalResult = result['Mbps'].describe()
    print("\n")
    print("Variance in Mbps results:")
    return finalResult
    
    
    
    

In [161]:
# list of unique tiers per ISP and tech

uniquetiers = """
SELECT DISTINCT mt.download,mt.ISP, mt.Technology,COUNT(DISTINCT httpmt.unit_id) as totalBoxes,

FROM `broadband-data.mba_data.curr_httpgetmt` as httpmt 
    INNER JOIN`broadband-data.mba_data.unit-profile-sept2020` as mt
    ON httpmt.unit_id=mt.unit_id

GROUP BY mt.download,mt.ISP, mt.Technology
HAVING totalBoxes >= 30
ORDER BY mt.download

"""
uniquetiers= client.query(uniquetiers).to_dataframe()
print("Unique download tiers with sample size >= 30: ")
print("\n")
print(uniquetiers)

Unique download tiers with sample size >= 30: 


    download              ISP Technology  totalBoxes
0        1.5      CenturyLink        DSL          65
1        3.0      CenturyLink        DSL          50
2        5.0  Cincinnati Bell        DSL          49
3        6.0         Frontier        DSL          36
4       10.0      CenturyLink        DSL          73
5       10.0       Windstream        DSL          56
6       12.0         Frontier        DSL          66
7       12.0      CenturyLink        DSL          66
8       12.0       Windstream        DSL          51
9       15.0       Windstream        DSL          46
10      18.0         Frontier        DSL          44
11      20.0      CenturyLink        DSL          57
12      25.0      CenturyLink        DSL          87
13      25.0          Comcast      Cable          57
14      25.0       Windstream        DSL          56
15      30.0  Cincinnati Bell        DSL          31
16      40.0      CenturyLink        DSL          

<h3>Sample Confidence Intervals </h3>

In [171]:
ispTierCI(100, "Comcast")

Getting results for 'Comcast' Cable download tier: 100 Mbps...
95% Confidence Interval (Mbps) (114.47810036163058, 115.2760764247807)
95th Consistent Speed for this tier and ISP is 93.93108


Variance in Mbps results:


count    2767.000000
mean      114.877088
std        10.703513
min         0.227976
25%       115.643784
50%       118.722288
75%       119.027348
max       121.017336
Name: Mbps, dtype: float64

In [172]:
ispTierCI(100, "Mediacom")

Getting results for 'Mediacom' Cable download tier: 100 Mbps...
95% Confidence Interval (Mbps) (128.5120460489106, 129.37855270840592)
95th Consistent Speed for this tier and ISP is 114.365784


Variance in Mbps results:


count    2221.000000
mean      128.945299
std        10.411933
min        25.194328
25%       129.430872
50%       132.226312
75%       133.252016
max       134.058552
Name: Mbps, dtype: float64

In [173]:
ispTierCI(100, "Frontier")

Getting results for 'Frontier' Fiber download tier: 100 Mbps...
95% Confidence Interval (Mbps) (97.32638190715919, 97.75491753340027)
95th Consistent Speed for this tier and ISP is 90.198064


Variance in Mbps results:


count    2288.000000
mean       97.540650
std         5.226453
min        16.816456
25%        97.192352
50%        99.184548
75%        99.908648
max       102.652296
Name: Mbps, dtype: float64

In [174]:
ispTierCI(100, "Charter")

Getting results for 'Charter' Cable download tier: 100 Mbps...
95% Confidence Interval (Mbps) (112.35758249630123, 113.12563374396845)
95th Consistent Speed for this tier and ISP is 105.683032


Variance in Mbps results:


count    3263.000000
mean      112.741608
std        11.188182
min         3.027848
25%       112.124068
50%       114.261576
75%       116.934292
max       142.214656
Name: Mbps, dtype: float64

In [175]:
ispTierCI(100, "Windstream")

Getting results for 'Windstream' DSL download tier: 100 Mbps...
95% Confidence Interval (Mbps) (95.84516999424982, 96.9408629048856)
95th Consistent Speed for this tier and ISP is 74.675136


Variance in Mbps results:


count    2082.000000
mean       96.393016
std        12.746724
min        11.921944
25%        87.517184
50%       100.328292
75%       104.824344
max       118.440064
Name: Mbps, dtype: float64

In [176]:
ispTierCI(300, "Comcast")

Getting results for 'Comcast' Cable download tier: 300 Mbps...
95% Confidence Interval (Mbps) (344.35260089128485, 346.8073282653128)
95th Consistent Speed for this tier and ISP is 288.584288


Variance in Mbps results:


count    3327.000000
mean      345.579965
std        36.107179
min         0.176704
25%       351.049800
50%       356.275248
75%       357.077624
max       359.690088
Name: Mbps, dtype: float64

In [177]:
ispTierCI(500, "Frontier")

Getting results for 'Frontier' Fiber download tier: 500 Mbps...
95% Confidence Interval (Mbps) (472.64551456416604, 477.2070546091673)
95th Consistent Speed for this tier and ISP is 350.141888


Variance in Mbps results:


count    4650.000000
mean      474.926285
std        79.331711
min        20.154576
25%       490.860192
50%       501.356104
75%       505.047896
max       569.141792
Name: Mbps, dtype: float64

In [178]:
ispTierCI(500, "Mediacom")

Getting results for 'Mediacom' Cable download tier: 500 Mbps...
95% Confidence Interval (Mbps) (545.6179244240025, 550.4778455815072)
95th Consistent Speed for this tier and ISP is 419.656928


Variance in Mbps results:


count    2541.000000
mean      548.047885
std        62.466411
min         1.255616
25%       561.026592
50%       570.112472
75%       571.327232
max       572.469672
Name: Mbps, dtype: float64

In [179]:
ispTierCI(400, "Charter")

Getting results for 'Charter' Cable download tier: 400 Mbps...
95% Confidence Interval (Mbps) (442.85228829950415, 446.88169592462043)
95th Consistent Speed for this tier and ISP is 294.94772


Variance in Mbps results:


count    3855.000000
mean      444.866992
std        63.802681
min         0.185472
25%       451.656928
50%       470.380192
75%       478.674856
max       482.058080
Name: Mbps, dtype: float64